In [ ]:
import json

catalog = json.loads(open("./catalog.json").read())

In [ ]:
import mysql.connector
import configparser

config = configparser.ConfigParser()
config.read("mysql.ini")

agg_db = mysql.connector.connect(
    host=config["mysql"]["host"],
    user=config["mysql"]["username"],
    password=config["mysql"]["password"],
    ssl_disabled=True,  # Disable SSL
)
agg_cursor = agg_db.cursor()

In [ ]:
# ------------------------------------------------
# ------------------------------------------------
# ------------- PROCESS WEAT CONTENT -------------
# ------------------------------------------------
# ------------------------------------------------

import re
import requests


def process_weat_pcex_items():
    print("connection to aggregate database established")

    prefix = "http://pawscomp2.sis.pitt.edu/pcex-authoring/preview/index.html?load="

    for _, item in enumerate(catalog):
        if not item["url"].startswith(prefix):
            continue
        iframe_url = item["url"][len(prefix) :].split("?")
        index = int(
            [p for p in iframe_url[1].split("&") if p.startswith("index=")][0].split(
                "="
            )[1]
        )
        content = requests.get(iframe_url[0]).json()[0]
        content = content["activityGoals"][index]

        metadata = dict()

        if content["fullyWorkedOut"] == False:
            metadata["distractors"] = []
            for line in content["distractorList"]:
                line = line["line"]
                del line["id"]
                del line["indentLevel"]
                if "number" in line and line["number"] > 0:
                    line["line_number"] = int(line["number"])
                del line["number"]
                metadata["distractors"].append(line)

        agg_cursor.execute(
            """INSERT INTO aggregate.ent_content_attrs (content_id, description, code, iframe_url, metadata) VALUES (%s, %s, %s, %s, %s, %s)""",
            (
                item["id"],
                content["goalDescription"],
                "\n".join(
                    [
                        l["content"]
                        for l in sorted(
                            content["lineList"],
                            key=lambda l: int(l["number"]),
                        )
                    ]
                ),
                item["url"],
                json.dumps(metadata) if len(metadata.keys()) > 0 else "",
            ),
        )

        print(f"processed content_id: {item['id']}")

    agg_db.commit()


# process_weat_pcex_items()